In [15]:
import sympy as sym
x = sym.Symbol('x')
y = sym.Symbol('y')
elem = 2*x + y
expression = (int(1.0) - elem)**2
expression = sym.expand(expression)
offset = 0
symbols = expression.free_symbols
for symbol in symbols:
    expression = expression.replace(symbol**2, symbol)

local_hubo = {}
for term in expression.as_ordered_terms():
    #print(term)
    coeff, vars = term.as_coeff_mul()
    
    if len(vars) == 0:
        print(coeff, vars)
        offset += coeff
    vars = [str(v) for v in vars]
    vars_tuple = tuple(sorted(vars))
    
#print(vars_tuple)

1 ()


In [4]:
import pickle
import dimod
from utils import solve_bqm_in_leap

hubo = None
with open('hubo.pkl', 'rb') as f:
    hubo = pickle.load(f)
print(len(hubo))
hubo, offset = hubo["hubo"], hubo["offset"]
print(len(hubo))
#new_hubo = {}

i = 0
for k, v in hubo.items():
    print(k, v)
    i += 1
    if i > 10:
        break
    


2
98112
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1x0', 'neg_1y1', 'neg_1z3') -2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1x0', 'neg_1y1', 'pos_1z3') 2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1x0', 'neg_1z3', 'pos_1y1') 2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1x0', 'pos_1y1', 'pos_1z3') -2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1y1', 'neg_1z3', 'pos_1x0') 2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1y1', 'pos_1x0', 'pos_1z3') -2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_1z3', 'pos_1x0', 'pos_1y1') -2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_2x0', 'neg_2y1', 'neg_2z3') -2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_2x0', 'neg_2y1', 'pos_2z3') 2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_2x0', 'neg_2z3', 'pos_2y1') 2
('neg_0x0', 'neg_0y1', 'neg_0z3', 'neg_2x0', 'pos_2y1', 'pos_2z3') -2


In [5]:
bqm = dimod.make_quadratic(hubo, 30, dimod.BINARY)
bqm.offset = 8.0
print("Offset: ", bqm.offset)
print("Linear: ", len(bqm.linear))
print("Quadratic: ", len(bqm.quadratic))

#sample, energy, sampleset = solve_bqm_in_leap(bqm, "Kerberos")

Offset:  8.0
Linear:  5720
Quadratic:  110944


In [6]:
import cplex
import time


def qubo_to_lp(bqm, identifier):
    cplex_problem = cplex.Cplex()
    cplex_problem.objective.set_sense(cplex_problem.objective.sense.minimize)
    variable_symbols = [str(var) for var in bqm.variables]
    cplex_problem.variables.add(names=variable_symbols, 
                                        types=[cplex_problem.variables.type.binary]*len(variable_symbols))

    linear_coeffs = bqm.linear
    obj_list = [(str(name), coeff) for name, coeff in linear_coeffs.items()]
    cplex_problem.objective.set_linear(obj_list)

    quadratic_coeffs = bqm.quadratic
    obj_list = [(str(name[0]), str(name[1]), coeff) for name, coeff in quadratic_coeffs.items()]
    cplex_problem.objective.set_quadratic_coefficients(obj_list)
    lp_file = "linear_program_files//" + str(identifier) + ".lp"
    cplex_problem.write(lp_file)
    return cplex_problem


def solve_with_cplex_solver(cplex_problem, offset, print_log = False):
    samplesets = {}
    if not print_log:
        cplex_problem.set_log_stream(None)
        cplex_problem.set_error_stream(None)
        cplex_problem.set_warning_stream(None)
        cplex_problem.set_results_stream(None)
    time_start = time.time()
    cplex_problem.solve()
    time_end = time.time()
    elapsed_time = time_end - time_start
    status = cplex_problem.solution.get_status_string()
    result = cplex_problem.solution.get_values()
    variables = cplex_problem.variables.get_names()
    value = cplex_problem.solution.get_objective_value() + offset
    samplesets["cplex"] = { "status": status, "energy": value, 
                                "time": elapsed_time, "result": dict(zip(variables, result)) }
    return samplesets

In [7]:
cplex_problem = qubo_to_lp(bqm, "matrix_mult")
samplesets = solve_with_cplex_solver(cplex_problem, bqm.offset)

Default row names c1, c2 ... being created.


In [ ]:
import numpy as np
from itertools import combinations
from utils import solve_bqm_in_leap

def flatten(data):
    if isinstance(data, tuple):
        for x in data:
            yield from flatten(x)
    else:
        yield str(data)

def pair_to_index(x, dim):
    return dim*(x[0] - 1) + x[1] - 1

def get_standard_tensor(dim):
    initial_tensor = np.zeros((dim**2, dim**2, dim**2))
    for c1 in range(1, dim + 1):
        for c2 in range(1, dim + 1):
            c = (c1, c2)
            for a2 in range(1, dim + 1):
                a = (c1, a2)
                b = (a2, c2)
                initial_tensor[pair_to_index(a, dim)][pair_to_index(b, dim)][pair_to_index(c, dim)] = 1
    return initial_tensor

def square_negative_sum(hubo, variables, offset):
    for v in variables:
        #v = tuple(sorted(list((flatten(var)))))
        if offset == 1:
            if v in hubo:
                hubo[v] -= 1
            else:
                hubo[v] = -1
        elif offset == 0:
            if v in hubo:
                hubo[v] += 1
            else:
                hubo[v] = 1
    combs = combinations(variables, 2)
    for pair in combs:
        v = tuple(sorted(list((flatten(pair)))))
        if v in hubo:
            hubo[v] += 2
        else:
            hubo[v] = 2
    return hubo

def construct_all_tensors(sample, dim, suggested_optimal):
    print(sample)
    positive_linear_vars = []
    tensors = []
    for i in range(suggested_optimal):
        x, y, z = [], [], []
        for j in range(dim**2):
            x.append(sample["pos_" + str(i) + "x" + str(j)] - sample["neg_" + str(i) + "x" + str(j)])
            y.append(sample["pos_" + str(i) + "y" + str(j)] - sample["neg_" + str(i) + "y" + str(j)])
            z.append(sample["pos_" + str(i) + "z" + str(j)] - sample["neg_" + str(i) + "z" + str(j)])
        tensors.append([x,y,z])
    return tensors

In [ ]:
tensors = construct_all_tensors(sample, 2, 7)

initial_tensor = get_standard_tensor(2)
for t in tensors:
    print(t)
    initial_tensor = np.mod(initial_tensor - np.tensordot(t[0], np.tensordot(t[1], t[2], axes=0), axes=0), 2)

# After substraction, the tensor should be the origo tensor (all entries are 0)
if np.count_nonzero(initial_tensor.flatten()) == 0:
    print("Success!")
else:
    print("No success")
    
print(initial_tensor)

{'neg_0x0': 1, 'neg_0x1': 0, 'neg_0x1*neg_0y0*pos_0z2': 0, 'neg_0x1*neg_0y1*pos_0z0': 0, 'neg_0x1*neg_0y1*pos_0z2': 0, 'neg_0x1*neg_0y2': 0, 'neg_0x1*neg_0y3*pos_0z2': 0, 'neg_0x1*neg_0z0*neg_0y2': 0, 'neg_0x1*neg_0z0*neg_0y3': 0, 'neg_0x1*neg_0z1*pos_0y3': 0, 'neg_0x1*pos_0y2': 0, 'neg_0x1*pos_0y3*pos_0z2': 0, 'neg_0x1*pos_0z0*neg_0y0': 0, 'neg_0x1*pos_0z1': 0, 'neg_0x1*pos_0z1*neg_0y0': 0, 'neg_0x1*pos_0z1*neg_0y2': 0, 'neg_0x1*pos_0z1*neg_0y3': 0, 'neg_0x1*pos_0z1*neg_0y3*pos_0x1': 0, 'neg_0x1*pos_0z1*neg_0z1*pos_0y0': 0, 'neg_0x1*pos_0z1*neg_3z1*neg_3x1': 0, 'neg_0x1*pos_0z1*neg_3z1*pos_3x1': 0, 'neg_0x1*pos_0z1*neg_4z1*neg_4x1': 0, 'neg_0x1*pos_0z1*neg_4z1*pos_4x1': 0, 'neg_0x1*pos_0z1*neg_5z1*neg_5x1': 0, 'neg_0x1*pos_0z1*neg_6z1*neg_6x1': 0, 'neg_0x1*pos_0z1*pos_0y0': 0, 'neg_0x1*pos_0z1*pos_0y1': 0, 'neg_0x1*pos_0z1*pos_0y2': 0, 'neg_0x1*pos_0z1*pos_0y3': 0, 'neg_0x1*pos_0z1*pos_3z1*pos_3x1': 0, 'neg_0x1*pos_0z1*pos_4x1*pos_4z1': 0, 'neg_0x1*pos_0z1*pos_4z1*neg_4x1': 0, 'neg_0x